<a href="https://colab.research.google.com/github/zooodung/Face_Image_Emotion_Classification/blob/JS/SVM_ViT_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import zipfile
import os

def unzip_file(zip_file_path, extract_dir):
    # 압축 파일 열기
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 압축 해제
        zip_ref.extractall(extract_dir)

# 압축 파일 경로와 추출할 디렉토리 경로 지정
zip_file_path = '/content/drive/MyDrive/Project/Data/img_preprocessed.zip'
extract_dir = '/content/img_p'

# 압축 파일 풀기
unzip_file(zip_file_path, extract_dir)

In [ ]:
train_data_dir = '/content/img_p/train'
val_data_dir = '/content/img_p/val'

In [ ]:
!pip install vit-keras
!pip install keras_applications
!pip install tensorflow_addons
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.0 MB/s eta 0:00:00


## Optuna 하이퍼 파라미터 최적화

In [ ]:
import optuna
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from vit_keras import vit
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# 데이터 전처리 및 특징 추출 함수
def preprocess_data(data_dir, datagen, target_size=(224, 224), image_size=224, model_name='vit_b16'):
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    # ViT 모델 직접 생성 (이미지 크기는 고정)
    if model_name == 'vit_b16':
        vit_model = vit.vit_b16(image_size=image_size, pretrained=True, include_top=False, pretrained_top=False)
    elif model_name == 'vit_l16':
        vit_model = vit.vit_l16(image_size=image_size, pretrained=True, include_top=False, pretrained_top=False)
    else:
        raise ValueError("Invalid model name. Choose either 'vit_b16' or 'vit_l16'.")

    # 이미지 크기 조정 추가
    input_tensor = tf.keras.layers.Input((target_size[0], target_size[1], 3))
    resized_tensor = tf.keras.layers.Resizing(image_size, image_size)(input_tensor)
    output_tensor = vit_model(resized_tensor)
    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

    embeddings = model.predict(generator)
    labels = generator.classes
    return embeddings, labels

# 데이터 증강 설정 (필요에 따라 변경 가능)
datagen = ImageDataGenerator(rescale=1./255)

# 데이터 로드 및 특징 추출 (ViT 모델은 vit_b16, 이미지 크기 224로 고정)
train_embeddings, train_labels = preprocess_data(train_data_dir, datagen)
val_embeddings, val_labels = preprocess_data(val_data_dir, datagen)

# 레이블 인코딩
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
val_labels = le.transform(val_labels)

# Optuna objective 함수 (SVM 하이퍼파라미터만 탐색)
def objective(trial):
    # SVM 하이퍼파라미터
    svm_c = trial.suggest_float('C', 1e-4, 1e4, log=True)
    svm_gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    # SVM 모델 학습 및 평가
    svm = SVC(kernel='rbf', C=svm_c, gamma=svm_gamma)
    svm.fit(train_embeddings, train_labels)
    val_predictions = svm.predict(val_embeddings)
    accuracy = accuracy_score(val_labels, val_predictions)
    return accuracy

# Optuna Study 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)  # 탐색 횟수 조절

# 최적 하이퍼파라미터 출력
print("Best SVM parameters:", study.best_params)
print("Best accuracy:", study.best_value)


Found 4324 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


136/136 [==============================] - 17s 108ms/step
Found 765 images belonging to 4 classes.
24/24 [==============================] - 5s 109ms/step


[I 2024-05-22 15:31:07,271] A new study created in memory with name: no-name-d7cca3c9-0277-40ff-bf46-b1e0afc2529b
[I 2024-05-22 15:31:17,554] Trial 0 finished with value: 0.3895424836601307 and parameters: {'C': 0.00025121970213525235, 'gamma': 'auto'}. Best is trial 0 with value: 0.3895424836601307.
[I 2024-05-22 15:31:21,620] Trial 1 finished with value: 0.8156862745098039 and parameters: {'C': 1297.2511230373525, 'gamma': 'scale'}. Best is trial 1 with value: 0.8156862745098039.
[I 2024-05-22 15:31:32,042] Trial 2 finished with value: 0.3908496732026144 and parameters: {'C': 0.0034087362455895182, 'gamma': 'scale'}. Best is trial 1 with value: 0.8156862745098039.
[I 2024-05-22 15:31:38,904] Trial 3 finished with value: 0.7294117647058823 and parameters: {'C': 0.12127555842453874, 'gamma': 'scale'}. Best is trial 1 with value: 0.8156862745098039.
[I 2024-05-22 15:31:42,779] Trial 4 finished with value: 0.8156862745098039 and parameters: {'C': 5801.210887286205, 'gamma': 'auto'}. Best

Best SVM parameters: {'C': 2.808856612710054, 'gamma': 'scale'}
Best accuracy: 0.8313725490196079


## SVM, ViT

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from vit_keras import vit
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# 데이터 전처리 및 특징 추출 함수
def preprocess_data(data_dir, datagen, target_size, image_size):
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    vit_model = vit.vit_b16(image_size=image_size, pretrained=True, include_top=False, pretrained_top=False)
    input_tensor = tf.keras.layers.Input((target_size[0], target_size[1], 3))
    resized_tensor = tf.keras.layers.Resizing(image_size, image_size)(input_tensor)
    output_tensor = vit_model(resized_tensor)
    model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

    embeddings = model.predict(generator)
    labels = generator.classes

    return embeddings, labels

datagen = ImageDataGenerator(rescale=1./255)

# 최적 하이퍼파라미터 적용
best_vit_image_size = 224
best_svm_c = 2.808856612710054
best_svm_gamma = 'scale'

train_embeddings, train_labels = preprocess_data(train_data_dir, datagen, target_size=(best_vit_image_size, best_vit_image_size), image_size=best_vit_image_size)
val_embeddings, val_labels = preprocess_data(val_data_dir, datagen, target_size=(best_vit_image_size, best_vit_image_size), image_size=best_vit_image_size)

le = LabelEncoder() # 인코딩
train_labels = le.fit_transform(train_labels)
val_labels = le.transform(val_labels)

# SVM 모델 학습
svm = SVC(kernel='rbf', C=best_svm_c, gamma=best_svm_gamma)
svm.fit(train_embeddings, train_labels)

# 모델 평가
val_predictions = svm.predict(val_embeddings)
accuracy = np.mean(val_predictions == val_labels)
print(f"Validation Accuracy (with best params): {accuracy:.4f}")


Found 4324 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


136/136 [==============================] - 17s 108ms/step
Found 765 images belonging to 4 classes.
24/24 [==============================] - 5s 108ms/step
Validation Accuracy (with best params): 0.8314


### 성능평가

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# 정확도 (Accuracy)
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Accuracy: {accuracy:.4f}")

# 정밀도 (Precision), 재현율 (Recall), F1 점수 (F1-score)
precision = precision_score(val_labels, val_predictions, average='weighted')
recall = recall_score(val_labels, val_predictions, average='weighted')
f1 = f1_score(val_labels, val_predictions, average='weighted')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# 분류 보고서 (Classification Report)
print("\nClassification Report:")
target_names = [str(label) for label in le.classes_]  # Convert labels to strings
print(classification_report(val_labels, val_predictions, target_names=target_names))


# 혼동 행렬 (Confusion Matrix)
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(val_labels, val_predictions)
print(conf_matrix)

Accuracy: 0.8314
Precision: 0.8299
Recall: 0.8314
F1-score: 0.8304

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.72      0.74       194
           1       0.89      0.93      0.91       192
           2       0.82      0.83      0.83       193
           3       0.84      0.84      0.84       186

    accuracy                           0.83       765
   macro avg       0.83      0.83      0.83       765
weighted avg       0.83      0.83      0.83       765


Confusion Matrix:
[[140   9  24  21]
 [  8 178   3   3]
 [ 25   1 161   6]
 [ 10  11   8 157]]


## model export

In [ ]:
import joblib

# Save the model
joblib.dump(svm, '/content/drive/MyDrive/Project/svm_vitembedding.joblib')

['/content/drive/MyDrive/Project/svm_vitembedding.joblib']

In [ ]:
loaded_svm = joblib.load('/content/drive/MyDrive/Project/svm_vitembedding.joblib')